# <span style="color:#DC143C">Negative Sampling 구현하기</span>

* 구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 라이브러리 임포트 및 공통 설정

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import random
import pickle, gc

# 컬럼명 매핑: 원본 한글 컬럼명을 영문 표준명으로 변환
# - '회원번호' -> 'USER': 사용자 식별자
# - 'ISBN' -> 'ITEM': 아이템(책) 식별자
COLUMN_MAPPING = {'회원번호': 'USER', 'ISBN': 'ITEM'}

# Negative Sampling 비율
# - 각 positive 샘플(실제 구매 기록) 당 생성할 negative 샘플(미구매 기록) 개수
N_NEG = 3

# 출력 파일명
# - 전처리 및 negative sampling이 완료된 데이터를 저장할 pickle 파일 경로
# - 포함 내용: train_df, test_df, train_item_meta, test_item_meta
OUTPUT_FILE = '/content/drive/MyDrive/4학기/추천시스템/실습-20251011/개인과제/book_train_test_1v3.pkl'

# 재현성을 위한 시드 고정
SEED = 42

#### 데이터 로드

In [3]:
# 원본 데이터 불러오기
# - low_memory=False: 대용량 데이터 처리
# - dropna(): 결측치 제거
# - query('ISBN != "-"'): ISBN이 "-"인 행 제외

ratings_df = pd.read_csv('/content/drive/MyDrive/4학기/추천시스템/실습-20251011/개인과제/book_transactions_8m.csv', encoding='cp949', low_memory=False).dropna(axis=0).query('ISBN != "-"')

#### 데이터 전처리

In [4]:
# 1. 컬럼명 변경 (표준화)
ratings_df = ratings_df.rename(columns=COLUMN_MAPPING)

# 2. ITEM 컬럼을 문자열로 통일 (숫자와 문자가 섞여있어서 에러 방지)
ratings_df['ITEM'] = ratings_df['ITEM'].astype(str)

# 3. 인코더 준비
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
cat_encoder = LabelEncoder()
author_encoder = LabelEncoder()
pub_encoder = LabelEncoder()

# 4. 모든 범주형 컬럼을 0부터 시작하는 숫자로 변환
ratings_df['USER'] = user_encoder.fit_transform(ratings_df['USER'])
ratings_df['ITEM'] = item_encoder.fit_transform(ratings_df['ITEM'])
ratings_df['category_id'] = cat_encoder.fit_transform(ratings_df['카테고리'])
ratings_df['author_id'] = author_encoder.fit_transform(ratings_df['작가'])
ratings_df['publisher_id'] = pub_encoder.fit_transform(ratings_df['출판사'])

# 5. 평점 컬럼 추가 (구매 기록은 모두 1점으로 처리)
ratings_df['RATING'] = 1

ratings_df.head()

,USER,일자,책제목,카테고리,작가,ITEM,출판사,출판일자,주문시간,수량,배송지,category_id,author_id,publisher_id,RATING
0,0,20140621,유형아작 중학수학 3-2 (2014년),중고등학습서,<김순우> 등저,13641,비상교육(구 비유와상징),20110801.0,23.0,1.0,서울특별시,20,2444,1192,1
1,0,20140621,개념+유형 중등수학 3-2 실력향상 파워 (2014년),중고등학습서,<박미정> 등저,15023,비상교육(구 비유와상징),20140301.0,23.0,1.0,서울특별시,20,5744,1192,1
2,0,20140621,내신특강 중학 수학 3-2 (2014년),중고등학습서,<김정우> 등저,6511,미래엔(대한교과서),20130530.0,23.0,1.0,서울특별시,20,2871,968,1
3,1,20140528,액셀 월드 (ACCEL WORLD) 1,만화/라이트노벨,<카와하라 레키> 저/<HIMA> 그림/<김완> 역,2860,서울문화사,20091110.0,20.0,1.0,충남,7,14048,1378,1
4,1,20140528,신약 어떤 마술의 금서목록 2,만화/라이트노벨,<카마치 카즈마> 저/<하이무라 키요타카> 그림/<김소연> 역,1650,대원씨아이(단행)(대원키즈),20120115.0,20.0,1.0,충남,7,14022,607,1


#### 메타데이터(Side Info) 저장 및 데이터 분할

In [5]:
# 1. 나중에 사용할 아이템 사이드 정보 미리 저장
#    각 아이템의 카테고리, 작가, 출판사 정보를 별도로 저장
item_side_info = ratings_df[['ITEM', 'category_id', 'author_id', 'publisher_id']].drop_duplicates('ITEM')

# 2. 전체 데이터를 train과 test로 분리
#    - USER, ITEM, RATING만 사용 (메타데이터는 나중에 필요시 merge)
train_df, test_df = train_test_split(
    ratings_df[['USER', 'ITEM', 'RATING']],
    test_size=0.3,
    random_state=SEED
)

# 3. 메모리 절약을 위해 원본 데이터 삭제
del ratings_df
gc.collect()

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

Train shape: (44625, 3)
Test shape: (19125, 3)


#### Negative Sampling 적용

In [6]:
# 목적: 각 positive 샘플(실제 구매한 책)마다 N_NEG개의 negative 샘플(안 산 책)을 추가
# 예: 사용자 A가 책 1권 샀다면 -> 1개 positive + 3개 negative = 총 4개 샘플 생성

# Step 1: 전체 아이템 목록 (중복 제거한 모든 책 ID)
print("Step 1/6: 전체 아이템 목록 확인...")
item_pool = set(train_df['ITEM'].unique())

# Step 2: 각 사용자가 구매한 아이템 세트 만들기
#    결과: USER | ITEM_interacted
#          1    | {책1, 책2, 책3}
#          2    | {책4, 책5}
print("Step 2/6: 사용자별 구매 아이템 세트 생성 중...")
interact_status = (
    train_df.groupby('USER')['ITEM']
    .apply(set)  # 리스트를 세트로 변환 (중복 제거 + 빠른 연산)
    .reset_index()
    .rename(columns={'ITEM': 'ITEM_interacted'})
)

# Step 3: 각 사용자가 구매하지 않은 아이템 세트 계산
#    전체 아이템 - 구매한 아이템 = 구매 안한 아이템
#    결과: ITEM_negative = {전체 책들} - {구매한 책들}
print("Step 3/6: 사용자별 negative 후보 생성 중...")
interact_status['ITEM_negative'] = interact_status['ITEM_interacted'].apply(
    lambda x: item_pool - x
)

# Step 4: 원본 데이터에 negative 후보 아이템 정보 붙이기
#    각 구매 기록마다 "이 사용자가 안 산 책 목록"이 추가됨
print("Step 4/6: Negative 후보 매핑 중...")
train_df = pd.merge(train_df, interact_status[['USER', 'ITEM_negative']], on='USER')

# Step 5: 각 구매 기록마다 N_NEG개의 negative 샘플 랜덤 선택
print(f"Step 5/6: 각 샘플당 {N_NEG}개 negative 샘플 랜덤 선택 중...")
try:
    # 정상 케이스: negative 후보가 N_NEG개 이상 있을 때
    train_df['ITEM_negative'] = train_df['ITEM_negative'].apply(
        lambda x: random.sample(list(x), N_NEG)
    )
except:
    # 예외 케이스: negative 후보가 N_NEG개보다 적을 때
    # 가능한 최대 개수만큼만 샘플링
    min_num = min(map(len, list(train_df['ITEM_negative'])))
    train_df['ITEM_negative'] = train_df['ITEM_negative'].apply(
        lambda x: random.sample(list(x), min_num)
    )

# Step 6: 최종 데이터 생성
#    각 구매 기록(positive)마다:
#      - 1개 positive: (user, item, rating=1.0)
#      - N_NEG개 negative: (user, random_item, rating=0.0)
print("Step 6/6: 전체 샘플 생성 중...")
users, items, ratings = [], [], []
for row in train_df.itertuples():
    # Positive 샘플 추가 (실제 구매한 아이템)
    users.append(getattr(row, 'USER'))
    items.append(getattr(row, 'ITEM'))
    ratings.append(float(getattr(row, 'RATING')))

    # Negative 샘플들 추가 (구매 안한 아이템들)
    for i in getattr(row, 'ITEM_negative'):
        users.append(getattr(row, 'USER'))
        items.append(i)
        ratings.append(float(0))  # negative는 0점

# 결과: 원본 데이터의 (1 + N_NEG)배 크기의 데이터프레임
train_df = pd.DataFrame({'USER': users, 'ITEM': items, 'RATING': ratings})

print(f"\n{'='*60}")
print(f"Negative Sampling 완료!")
print(f"{'='*60}")
print(f"  Positive samples: {(train_df['RATING'] == 1).sum():,}")
print(f"  Negative samples: {(train_df['RATING'] == 0).sum():,}")
print(f"  Ratio: 1:{(train_df['RATING'] == 0).sum() / (train_df['RATING'] == 1).sum():.1f}")
print(f"{'='*60}")
train_df.head(10)

Step 1/6: 전체 아이템 목록 확인...
Step 2/6: 사용자별 구매 아이템 세트 생성 중...
Step 3/6: 사용자별 negative 후보 생성 중...
Step 4/6: Negative 후보 매핑 중...
Step 5/6: 각 샘플당 3개 negative 샘플 랜덤 선택 중...
Step 6/6: 전체 샘플 생성 중...

Negative Sampling 완료!
  Positive samples: 44,625
  Negative samples: 133,875
  Ratio: 1:3.0


,USER,ITEM,RATING
0,242,16185,1.0
1,242,1586,0.0
2,242,8682,0.0
3,242,22087,0.0
4,2990,23683,1.0
5,2990,5403,0.0
6,2990,4972,0.0
7,2990,1215,0.0
8,6552,1284,1.0
9,6552,13089,0.0


#### 아이템 메타데이터 분리

In [7]:
# Train과 Test에 사용된 아이템의 메타데이터를 별도로 분리
# 목적: train_df, test_df는 USER, ITEM, RATING만 포함하고
#       메타데이터는 필요시 merge해서 사용

# Train 아이템 메타데이터
# - train_df에 실제로 등장하는 아이템들의 메타정보만 추출
train_items = set(train_df['ITEM'].unique())
train_item_meta = item_side_info[item_side_info['ITEM'].isin(train_items)].copy()
train_item_meta = train_item_meta.sort_values('ITEM').reset_index(drop=True)

# Test 아이템 메타데이터
# - test_df에 실제로 등장하는 아이템들의 메타정보만 추출
test_items = set(test_df['ITEM'].unique())
test_item_meta = item_side_info[item_side_info['ITEM'].isin(test_items)].copy()
test_item_meta = test_item_meta.sort_values('ITEM').reset_index(drop=True)

print("Train Item Metadata:")
print(f"  Shape: {train_item_meta.shape}")
print(train_item_meta.head())

print("\nTest Item Metadata:")
print(f"  Shape: {test_item_meta.shape}")
print(test_item_meta.head())

Train Item Metadata:
  Shape: (20944, 4)
   ITEM  category_id  author_id  publisher_id
0     1           18      16531          2305
1     3           20       9774          2031
2     4           18      16624            51
3     5           18      16149           158
4     6           20       9774          2031

Test Item Metadata:
  Shape: (11412, 4)
   ITEM  category_id  author_id  publisher_id
0     0           18      16239           684
1     1           18      16531          2305
2     2           20       9774          2031
3     4           18      16624            51
4     8           18      16572           734


#### 데이터 저장

In [8]:
# Train, Test 데이터와 아이템 메타데이터를 리스트 형태로 저장
# 구조:
#   - train_df: USER, ITEM, RATING
#   - test_df: USER, ITEM, RATING
#   - train_item_meta: ITEM, category_id, author_id, publisher_id
#   - test_item_meta: ITEM, category_id, author_id, publisher_id

with open(OUTPUT_FILE, 'wb') as f:
    pickle.dump([train_df, test_df, train_item_meta, test_item_meta], f)

print(f"\n저장 완료: {OUTPUT_FILE}")
print(f"  - train_df: {train_df.shape}")
print(f"  - test_df: {test_df.shape}")
print(f"  - train_item_meta: {train_item_meta.shape}")
print(f"  - test_item_meta: {test_item_meta.shape}")


저장 완료: /content/drive/MyDrive/4학기/추천시스템/실습-20251011/개인과제/book_train_test_1v3.pkl
  - train_df: (178500, 3)
  - test_df: (19125, 3)
  - train_item_meta: (20944, 4)
  - test_item_meta: (11412, 4)


# <span style="color:#DC143C">End</span>